# 0. 注意

1. Google Colab の制約

Google Colabで長時間処理を行う場合は、必ずChromeで実行してください。
その際に、事前にAutoRefreshプラグインを有効にして90分以内にリフレッシュがかかるようにしておいてください。

2. Google Colab 環境

Google Colabが稼働するコンテナはGoogleの都合で勝手にバージョンアップします。
このため、古いcudaライブラリ前提のTensorflowバージョンが、突然動作しなくなることがあります。

* [Model trained by Google Colab cannot run in Donkey Car](https://medium.com/@jonathantse/model-trained-by-google-colab-cannot-run-in-donkey-car-5d5347674b77)

# 1. 必要なパッケージのインストール

以下のセルをそのまま実行する。

In [18]:
# donkeycar インストール
%cd /content/
!rm -rf donkeycar
!git clone https://github.com/autorope/donkeycar
%cd /content/donkeycar
!git checkout master
!pip install -e .[pc]
!pip install tensorflow-gpu==1.15.0
%cd /content/
!rm -rf caterpillar
!git clone https://github.com/coolerking/caterpillar

/content
Cloning into 'donkeycar'...
remote: Enumerating objects: 10958, done.
remote: Total 10958 (delta 0), reused 0 (delta 0), pack-reused 10958
Receiving objects: 100% (10958/10958), 58.79 MiB | 23.49 MiB/s, done.
Resolving deltas: 100% (6801/6801), done.
/content/donkeycar
Branch 'master' set up to track remote branch 'master' from 'origin'.
Switched to a new branch 'master'
Obtaining file:///content/donkeycar
  Found existing installation: donkeycar 3.1.1
    Can't uninstall 'donkeycar'. No files were found to uninstall.
  Running setup.py develop for donkeycar
     |████████████████████████████████| 411.5MB 40kB/s 
     |████████████████████████████████| 512kB 42.4MB/s 
     |████████████████████████████████| 3.8MB 42.8MB/s 
  Found existing installation: tensorflow-estimator 1.13.0
    Uninstalling tensorflow-estimator-1.13.0:
      Successfully uninstalled tensorflow-estimator-1.13.0
  Found existing installation: tensorboard 1.13.1
    Uninstalling tensorboard-1.13.1:
      S

# 2. Donkeycar上のdataをアップロード

* Donkeycar上でログインし以下のコマンドを実行する

```bash
cd projects/caterpillar
tar cvfz datas.tar.gz ./data
```

* PC上から以下のコマンドを実行する

```bash
cd $HOME/projects/caterpillar
scp pi:raspberry@192.168.8.XXX:/home/pi/projects/datas.tar.gz .
```

* Colabの左のペインを開いて「ファイル」を選択して/contentを開いた状態にする
* 「アップロード」を選択し、さきほどの datas.tar.gz を選択
* アップロードが完了したら以下のセルを実行する

In [19]:
%cd /content/caterpillar
!rm -rf data
!tar xvfz ../datas_20191122_01.tar.gz . 

/content/caterpillar
./data/
./data/tub_1_19-11-22/
./data/tub_1_19-11-22/record_5716.json
./data/tub_1_19-11-22/record_976.json
./data/tub_1_19-11-22/600_cam-image_array_.jpg
./data/tub_1_19-11-22/4320_cam-image_array_.jpg
./data/tub_1_19-11-22/record_3825.json
./data/tub_1_19-11-22/6411_cam-image_array_.jpg
./data/tub_1_19-11-22/2437_cam-image_array_.jpg
./data/tub_1_19-11-22/1932_cam-image_array_.jpg
./data/tub_1_19-11-22/2506_cam-image_array_.jpg
./data/tub_1_19-11-22/record_3664.json
./data/tub_1_19-11-22/4000_cam-image_array_.jpg
./data/tub_1_19-11-22/4393_cam-image_array_.jpg
./data/tub_1_19-11-22/record_5068.json
./data/tub_1_19-11-22/record_4151.json
./data/tub_1_19-11-22/record_1026.json
./data/tub_1_19-11-22/record_1155.json
./data/tub_1_19-11-22/3542_cam-image_array_.jpg
./data/tub_1_19-11-22/record_5488.json
./data/tub_1_19-11-22/record_3364.json
./data/tub_1_19-11-22/record_1008.json
./data/tub_1_19-11-22/record_499.json
./data/tub_1_19-11-22/4703_cam-image_array_.jpg
./d

# 3. 学習モデルファイルの作成

* python実行行のlinearやTubディレクトリパス、モデルファイルパスを環境にあわせて修正
* セルを実行する
* 実行完了したら、左のペインの「ファイル」を選択し、/content/caterpillarを開く
* 拡張子.h5のファイルを選択し、ダウンロード
* ダウンロードしたファイルをPC上から以下の例のようにscpアップロードする

```bash
scp linear.h5 pi:raspberry@192.168.X:/home/pi/projects/caterpillar/
```

* Donkeycar上で以下のコマンドを実行し、運転開始

```bash
cd ~/projects/caterpillar
python manage_cat.py drive --type linear --model linear.h5
```

* 終了する場合は、Ctrl+Cを押下する

In [21]:
%cd /content/caterpillar/
!python manage_cat.py train --type linear --tub data/tub_1_19-11-22 --model linear.h5

/content/caterpillar
using donkey v3.1.1 ...
loading config file: /content/caterpillar/config.py
loading personal config over-rides

config loaded
"get_model_by_type" model Type is: linear
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
training with model type <class 'donkeycar.parts.keras.KerasLinear'>
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img_in (InputLayer)             [(None, 120, 160, 3) 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 58, 78, 24)   1824        img_in[0][0]                     
__________________________________________________________________________________________________
dropout (Dropout)               (None, 58, 78, 24)  

# A. 可視化画像の作成

> 可視化画像は、linear、categoricalモデルで作成可能

* 環境やモデルに合わせて以下のセルを修正
* セルを実行する
* 実行が終了したら、左のペインの「ファイル」を選択し、/content/caterpillarを開く
* 完成した拡張子.m4ファイルをダウンロードする


In [22]:
%cd /content/
!pip install keras-vis
!git clone https://github.com/coolerking/keras-vis
!pip install -e keras-vis

/content
Cloning into 'keras-vis'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 97 (delta 1), reused 97 (delta 1), pack-reused 0
Unpacking objects: 100% (97/97), done.
Obtaining file:///content/keras-vis
  Found existing installation: keras-vis 0.4.1
    Uninstalling keras-vis-0.4.1:
      Successfully uninstalled keras-vis-0.4.1
  Running setup.py develop for keras-vis


In [23]:
%cd /content/caterpillar
!donkey makemovie --tub data/tub_1_19-11-22 --out linear.mp4 --type linear --model linear.h5 --start 1 --end 200 --salient

/content/caterpillar
using donkey v3.1.1 ...
Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3260416/45929032 bytes (7.1%)7233536/45929032 bytes (15.7%)10838016/45929032 bytes (23.6%)14311424/45929032 bytes (31.2%)17817600/45929032 bytes (38.8%)21430272/45929032 bytes (46.7%)25165824/45929032 bytes (54.8%)28884992/45929032 bytes (62.9%)32768000/45929032 bytes (71.3%)36470784/45929032 bytes (79.4%)40386560/45929032 bytes (87.9%)43925504/45929